In [1]:
import pandas as pd
import numpy as np
import re
import string
from nltk.corpus import stopwords
import nltk 
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

2023-11-19 02:43:28.827589: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
dataset = '/Users/andrii.p/Downloads/train.csv'

In [3]:
data = pd.read_csv(dataset)

In [4]:
data.head()

,sms,label
0,"Go until jurong point, crazy.. Available only ...",0
1,Ok lar... Joking wif u oni...\n,0
2,Free entry in 2 a wkly comp to win FA Cup fina...,1
3,U dun say so early hor... U c already then say...,0
4,"Nah I don't think he goes to usf, he lives aro...",0


In [5]:
data_to_clear = data['sms'].values

In [6]:
nltk.download('stopwords')
stop_words = stopwords.words("english")


[nltk_data] Error loading stopwords: <urlopen error [SSL:
[nltk_data]     CERTIFICATE_VERIFY_FAILED] certificate verify failed:
[nltk_data]     unable to get local issuer certificate (_ssl.c:997)>


In [7]:
def clean_text(text):
    text = text.lower()
    text = re.sub("@\S+", " ", text)
    text = re.sub("https*\S+", " ", text)
    text = re.sub("#\S+", " ", text)
    text = re.sub("\d", " ", text)
    text = re.sub('[%s]' % re.escape(string.punctuation), ' ', text)
    text = re.sub('\n', ' ', text)
    text = re.sub('\s{2,}',' ', text)
    text = ' '.join([word for word in text.split(' ') if word not in stop_words])
    return text

In [8]:
clean_data = []
for text in data_to_clear:
     clean_data.append(clean_text(text))

In [9]:
data['sms']=clean_data

In [10]:
data.head()

,sms,label
0,go jurong point crazy available bugis n great ...,0
1,ok lar joking wif u oni,0
2,free entry wkly comp win fa cup final tkts st ...,1
3,u dun say early hor u c already say,0
4,nah think goes usf lives around though,0


In [11]:
X = data["sms"]
y = data["label"]

In [12]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [13]:
tokenizer = tf.keras.preprocessing.text.Tokenizer()
tokenizer.fit_on_texts(X_train.tolist())


In [14]:
X_train = tokenizer.texts_to_sequences(X_train)
X_test = tokenizer.texts_to_sequences(X_test)

In [15]:
max_sequence_length = 100
X_train = tf.keras.preprocessing.sequence.pad_sequences(X_train, maxlen=max_sequence_length)
X_test = tf.keras.preprocessing.sequence.pad_sequences(X_test, maxlen=max_sequence_length)

In [16]:
model = tf.keras.models.Sequential()
model.add(tf.keras.layers.Embedding(input_dim=10000, output_dim=64))
model.add(tf.keras.layers.LSTM(128))
model.add(tf.keras.layers.Dense(1, activation='sigmoid'))

In [17]:
model.compile(optimizer="adam",
              loss="binary_crossentropy",
              metrics=["accuracy"])

In [28]:
model.fit(X_train, 
          y_train, 
          epochs=3,
          batch_size=128,
          validation_split=0.2)

Epoch 1/3
28/28 [==============================] - 4s 135ms/step - loss: 7.6750e-04 - accuracy: 1.0000 - val_loss: 0.0708 - val_accuracy: 0.9832
Epoch 2/3
28/28 [==============================] - 3s 123ms/step - loss: 6.0995e-04 - accuracy: 1.0000 - val_loss: 0.0705 - val_accuracy: 0.9854
Epoch 3/3
28/28 [==============================] - 3s 122ms/step - loss: 4.9729e-04 - accuracy: 1.0000 - val_loss: 0.0745 - val_accuracy: 0.9843


In [26]:
Y_pred = model.predict(X_test)

35/35 [==============================] - 1s 14ms/step


In [22]:
Y_pred = np.round(Y_pred)

In [23]:
print(accuracy_score(y_test, Y_pred))

0.9811659192825112
